In [1]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

In [2]:
from vae import VAE
from trainer import Trainer

In [3]:
# Device config 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


In [4]:
test = VAE(50, 250, 6, 25)

total_params = sum(p.numel() for p in test.parameters())
trainable_params = sum(p.numel() for p in test.parameters() if p.requires_grad)
print(trainable_params)


1563550


In [6]:
# Dataset import 
# Windows path
# data = pd.read_csv('A:/csv_hus/psam_hus_pus_filtered.csv')
# WSL path
data = pd.read_csv('/mnt/a/csv_hus/psam_hus_pus_filtered.csv', dtype='int32')

ValueError: invalid literal for int() with base 10: '2018HU0000147'

In [6]:
print(data.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11106182 entries, 0 to 11106181
Data columns (total 51 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   SERIALNO                   object
 1   TEN:owned or mortgaged     int64 
 2   TEN:rented                 int64 
 3   HHL:english                int64 
 4   HHL:spanish                int64 
 5   HHL:other indo-european    int64 
 6   VEH:no vehicles            int64 
 7   VEH:1 vehicle              int64 
 8   VEH:2 vehicles             int64 
 9   VEH:3 vehicles             int64 
 10  VEH:4 or more vehicles     int64 
 11  R18:no                     int64 
 12  R18:yes                    int64 
 13  R65:no                     int64 
 14  R65:yes                    int64 
 15  HINCP:under 5k             int64 
 16  HINCP:5k-10k               int64 
 17  HINCP:10k-15k              int64 
 18  HINCP:15k-20k              int64 
 19  HINCP:excluded             int64 
 20  HINCP:25k-35k         

In [7]:
print(data.values)

: 

In [6]:
# Data loader 

data = torch.tensor(data.values, dtype=torch.float32)
data_loader = DataLoader(data, batch_size=32, shuffle=True)

: 

In [7]:
model = VAE(50, 250, 6, 25)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

trainer = Trainer(model, optimizer, device)

In [8]:
# train the model 
trainer.train(data_loader, 10)

KeyError: 98468